# Introduction

Computer Vision is a fields of computer science that focuses on enabling artificial systems to extract information from images and its variants (e.g.: video sequences, views from multiple cameras, multi-dimensional data from a 3D scanner, medical scanning devices, etc.). It makes use of algorithmic models that allow a computer to "teach" itself the context of visual data, learning the patterns that distinguish an image from another. Computational vision is rapidly gaining popularity for automated AI vision inspection, remote monitoring, and automation.

Computer Vision has become the backbone of numerous practical applications that significantly impact our daily lives and companies across industries, from retail to security, healthcare, construction, automotive, manufacturing, logistics, and agriculture.

In this scenario, one of the most groundbreaking approaches for Computer Vision is the **You Only Look Once** (YOLO) models family.

# Development History

The first version of YOLO(You Look Only Once) was conceived by Joseph Redmon at al. in their 2015 [paper](https://arxiv.org/abs/1506.02640), and represented a groundbreaking approach in Computer Vision, particularly in object *detection tasks.* At the, time the conventional object detection frameworks (e.g.: RCNN) relied on a two-step approach: for a given image, one model is responsible for extraction of regions of objects, and a second model is responsible for classification and refinement of localization of objects. YOLOv1 (how the first version became known) challenged this convention by proposing a single neural network that predictions bounding boxes and class probabilities directly from full images in one evaluation. The approach significantly increased the speed of detection, making real-time object detection feasible.

![RCNN pipeline](imgs/rcnn-pipeline.png)
*RCNN's multi-stage detection. Credits: RCNN original [paper](https://arxiv.org/abs/1311.2524).*

![YOLOv1 pipeline](imgs/yolov1-pipeline.png)
*YOLOv1 unified detection. Credits: YOLOv1 original [paper](https://arxiv.org/abs/1506.02640).*

Following the initial release, the YOLO architecture underwent several iterations and improvements, leading to version like YOLOv2 ([YOLO9000](https://arxiv.org/abs/1612.08242)), [YOLOv3](https://arxiv.org/abs/1804.02767), and further, each introducing enhancements in speed, accuracy, and the ability to detect smaller objects. [YOLOv4](https://arxiv.org/abs/2004.10934), introduced by Alexey Bochkovskiy, focused on optimizing the speed, accuracy trade-off, making it highly efficient without specialized hardware.

The [Ultralytics](https://www.ultralytics.com/) team contributed significantly to the YOLO legacy with their [YOLOv5](https://docs.ultralytics.com/yolov5/) model, which brought improvements in terms of simplicity, speed, and performance. They continued this trend with the development of [YOLOv6](https://arxiv.org/abs/2209.02976) and [YOLOv8](https://docs.ultralytics.com/), which incorporates advanced features and improving upon the accuracy and efficiency of its predecessors.

YOLOv8 also supports a full range of vision AI tasks, including detection, segmentation, pose estimation, tracking and classification. This versatility allows users to leverage YOLOv8's capabilities across diverse application and domains.

Currently, [YOLO-NAS](https://github.com/Deci-AI/super-gradients/blob/master/YOLONAS.md) and [YOLOv9](https://arxiv.org/abs/2402.13616) were conceived with remarkable improvements in efficiency, accuracy and adaptability.

# Architecture

<img src="imgs/yolov8_architecture.jpg" alt="YOLOv8 Architecture" width=800>*Credits: GitHub user [RangeKing](https://github.com/RangeKing) ([original post](https://github.com/ultralytics/ultralytics/issues/189))*</img>

The YOLOv8 architecture was designed to perform object detection tasks with high efficiency and accuracy. While maintaining the core principle of performing object detection in a single pass through the network, YOLOv8 introduces several key improvements and features to enhance performance, incorporating advanced techniques such as:
- **Cross-stage Partial Networks (CSPNet):** A backbone designed to reduce redundancy in network layers, reducing it's complexity and improving learning efficiency and model scalability without compromising performance.
- **Path Aggregation Network (PANet):** An architecture that enahnces feature extraction and integration ensuring rich semantic information is carried through the network for accurate detection.
- **Spatial Pyramid Pooling (SPP):** A pooling strategy that increases the network's robustness to object scale variations, improving detection of objects of various sizes.

Additionally, YOLOv8 employs advanced data augmentation techniques and loss functions ([CIoU](https://arxiv.org/abs/1911.08287) and [DFL](https://arxiv.org/abs/2006.04388)) to fine-tune the model's performance further, ensuring it remains robust against a wide variety of images and scenarios.

# Key Features

YOLOv8 support a versatile range of Computer Vision tasks and pre-trained models for each:
- **Image classification**, with models pre-trained in the [ImageNet dataset](https://www.image-net.org/).
- **Object detections**, **object segmentation** and **human pose estimation**, with models pre-trained in the [COCO dataset](https://cocodataset.org/#home).

Additionally, there are model variatesion for each of these tasks, each variation targeted to run on systems with different hardware specifications.

# Imports

# Object Detection

# Image Classification

# Semantic Segmentation

# Human Pose Estimation